In [ ]:
import pymysql
import pandas as pd

class MySQL():
    def __init__(self,passwd):
        self.conn = pymysql.connect(host='localhost', unix_socket=None, user='corpnet', passwd=passwd, db='2017orbis') # new data
        #self.conn = pymysql.connect(host='localhost', unix_socket=None, user='corpnet', passwd=passwd, db='2015-Sept') # old data
        self.cur = self.conn.cursor()

    def close(self):
        self.cur.close()
        self.conn.close()

    def getFirstLevel(self,shareholder,fileSave="./data_companyNetworks/subsidiaries_edges.csv"):
        # get BvD of all the companies with > 50 direct or total.
        with open(fileSave,"a+") as f:

            # old data
            #self.cur.execute("SELECT ownership.BvD_ID,ownership.directOwn,ownership.totalOwn " \
            #                    "FROM ownership " \
            #                "WHERE ownership.`BvD_ID_Shareholder` = \"{0}\" AND (ownership.`directOwn` > 50 OR ownership.`totalOwn` > 50)".format(shareholder))
            
            # new data            
            self.cur.execute("SELECT o.SubsidiaryBvD, o.direct, o.total " \
                                "FROM ownershipCurrent o " \
                            "WHERE o.ShareholderBvD = \"{0}\" AND (o.directNumber > 50 OR o.totalNumber > 50)".format(shareholder))
            core = []
            for r in self.cur:
                #print(("{}\t"*(len(r)+1)).format(shareholder,*r)+"\n")
                f.write(("{}\t"*(len(r)+1)).format(shareholder,r[0],100,*r[2:]).rstrip()+"\n")
                core.append(r[0])
        return set(core)

    def getSubsidiaries(self,shareholder,directOwner=5.,fileSave="./data_companyNetworks/subsidiaries_edges.csv",coreCompany = False):
        with open(fileSave,"a+") as f:
            
            # old data
            #self.cur.execute("SELECT ownership.BvD_ID,ownership.directOwn,ownership.totalOwn " \
            #                "FROM ownership " \
            #                "WHERE ownership.`BvD_ID_Shareholder` = \"{0}\" AND (ownership.`directOwn` >= {1})".format(shareholder,directOwner))

            # new data
            self.cur.execute("SELECT o.SubsidiaryBvD, o.direct, o.total " \
                            "FROM ownershipCurrent o " \
                            "WHERE o.ShareholderBvD = \"{0}\" AND (o.directNumber >= {1})".format(shareholder,directOwner))


            if coreCompany: shareholder = coreCompany
            subsidiaries = []
            for r in self.cur:
                #print(("{}\t"*(len(r)+1)).format(shareholder,*r)+"\n")
                f.write(("{}\t"*(len(r)+1)).format(shareholder,*r).rstrip()+"\n")
                subsidiaries.append(r[0])

        return set(subsidiaries)

    def getNodeInformation(self,companiesList,fileSave="./data_companyNetworks/subsidiaries_nodes.csv"):
        with open(fileSave,"w+") as f:
            r = ["Id", "Label", "City", "Country", "Sector", "Revenue", "MainExchange", "Assets", "Employees", "State"] #, "CashFlow", "ShareholderFunds", "P/L"]
            f.write(("{}\t"*(len(r))).format(*r)+"\n")
            for company in companiesList:
                
                # old data
                #self.cur.execute("SELECT companies_all_info.`BvD_ID:Id`,companies_all_info.companyName, companies_all_info.city, companies_all_info.country, companies_all_info.BvD_majorSector, " \
                #                 "companies_all_info.`operatingRevenue:long`, companies_all_info.`mainExchange`, companies_all_info.`totalAssets:long`, companies_all_info.`employees:int`, companies_all_info.`cashFlow:long`, " \
                #                 "companies_all_info.`shareholdersFunds:long`,companies_all_info.`P_L_beforeTax:int` " \
                #                 "FROM companies_all_info " \
                #                "WHERE companies_all_info.`BvD_ID:Id` = \"{0}\";".format(company))

                # new data
                self.cur.execute("SELECT c.BvD, c.companyName, c.city, c.iso2, c.letterNACE, " \
                                 "c.operatingRevenue, c.mainExchange, c.currentAssets, c.numberEmployees, CASE WHEN  ( c.typeEntity = 'Public authority/State/Government'  OR  c.letterNACE = 'O' ) THEN 1 else 0 END AS State " \
                                 "FROM recentFinancials c " \
                                "WHERE c.BvD = \"{0}\";".format(company))

                for r in self.cur:
                    #print(("{}\t"*(len(r)+1)).format(shareholder,*r)+"\n")
                    f.write(("{}\t"*(len(r))).format(*r)+"\n")


    def getSeveralLevelsSubsidiaries(self,nameCompany,initCompany,directOwner=5.,numberLayers = 10):
        fileSave = "./data_companyNetworks/subsidiaries{0}_{1}_edges.csv".format(nameCompany,numberLayers)
        core = self.getFirstMultiLevel(initCompany,fileSave=fileSave)

        allSubsidiaries = listStudy = set(core)
        for i in range(numberLayers):
            print("Layer: ",i+1)
            layerSubsidiaries = set(initCompany)
            for IDcompany in listStudy:
                if IDcompany in core:
                    subsidiaries = self.getSubsidiaries(IDcompany,directOwner,fileSave,coreCompany=initCompany)
                else:
                    subsidiaries = self.getSubsidiaries(IDcompany,directOwner,fileSave)
                layerSubsidiaries = layerSubsidiaries | subsidiaries

            listStudy = layerSubsidiaries - allSubsidiaries
            allSubsidiaries = allSubsidiaries | layerSubsidiaries


        #Drop duplicates (initCompany -> core1 -> core2 is counted twice as initCompany -> core2 since we're considering all of them as initCompany)
        df = pd.read_csv(fileSave,sep="\t",header=None,index_col=None,na_values=["None"])
        df = df.sort_values(by=[0,1,2,3])
        df.drop_duplicates(subset=[0,1],inplace=True)

        df.columns=["Source","Target","Weight","WeightTotal"]
        df.to_csv(fileSave,sep="\t",index=None)

        listCompanies = set(df["Source"]) | set(df["Target"])
        fileSave = "./data_companyNetworks/subsidiaries{0}_{1}_nodes.csv".format(nameCompany,numberLayers)
        self.getNodeInformation(listCompanies,fileSave)

if __name__ == "__main__":
    import sys, os
    listArg = sys.argv

    #listArg = ["_","Blackrock","US320174431",2]
    if len(listArg) == 4:
        companyName, companyID,layers = str(listArg[1]),str(listArg[2]),int(listArg[3])
    else:
        companyName = str(input("Insert company name: "))
        companyID = str(input("Insert BvD ID: "))
        layers = int(input("Insert number of Layers: "))

    if not os.path.exists("./data_companyNetworks"):
        print("Creating directory data_companyNetworks")
        os.makedirs("./data_companyNetworks")


    a = MySQL()
    a.getSeveralLevelsSubsidiaries(companyName,companyID,numberLayers=layers)
    a.close()

